In [ ]:
Method/tool                        Improves training speed             Optimizes memory utilization
Batch size choice                           Yes                                   Yes
Gradient accumulation  #减速                No                                    Yes
Gradient checkpointing#减速20%              No                                    Yes
Mixed precision training #fp16=True,        Yes                                   No
Optimizer choice                            Yes                                   Yes
Data preloading                             Yes                                    No
DeepSpeed Zero                              No                                    Yes
torch.compile    这个有设备要,提速30%       Yes                                    No
Parameter-Efficient Fine Tuning (PEFT)      No                                     Yes

# 数据，难点就在于数据的处理
# 和如何去进行任务评估

In [ ]:
from datasets import load_dataset
from torchvision.transforms import ColorJitter,RandomResizedCrop, Compose, Normalize, ToTensor
from transformers import AutoImageProcessor
dataset = load_dataset("scene_parse_150",
                  #split="train[:50]",选择train中的前50个样本
                 )
dataset= dataset.train_test_split(test_size=0.2)#需要对整体数据集进行切分
train_data = dataset['train']
test_data=dataset['test']

In [ ]:
classname = train_data.features['label'].names
id2label = {int(k): v for k, v in enumerate(classname)}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

In [ ]:
#增益图片
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)
# 传入的就是列表，这个调整图片大小，还有标准化，不做增益
checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
#传入一个整合的batch，所以要遍历每一个样本进行增益
#而增益又是 自定义函数+image_processor的形式，所以这里是让数据格式满足img_processor可以使用的形式
#这里还是dataset，不是dataloader

#这是一个字典，每个value已经被整整合了
#所以最好进行遍历的方式处理
def train_transforms(example_batch):
    #example_batch  ={‘image':[],'label':[]}
    example_batch['pixel_values']=[_transforms(x.convert("RGB")) for x in example_batch['image']]
    del example_batch['image']
    return example_batch

def val_transforms(example_batch):
    example_batch['pixel_values']=[_transforms(x.convert("RGB")) for x in example_batch['image']]
    del example_batch['image']
    return example_batch

In [ ]:
#函数必须接受一个字典，字典的健就是一个样本中的一样，只是它的值是列表，需要遍历每一个进行增益
#
#data = train_ds.map(func)这种形式，func必须是处理一个样本的，返回一个字典
# function (`callable`): with one of the following signature:
#    - `function(example: Dict) -> Union[Dict, Any]` if `batched=False` and `with_indices=False`
#    - `function(example: Dict, indices: int) -> Union[Dict, Any]` if `batched=False` and `with_indices=True`
#    - `function(batch: Dict[List]) -> Union[Dict, Any]` if `batched=True` and `with_indices=False`
#    - `function(batch: Dict[List], indices: List[int]) -> Union[Dict, Any]` if `batched=True` and `with_indices=True`

#. The transform is applied on-the-fly on batches
train_ds=train_ds.with_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
#这个是常规的collate_fn函数，就是将许多个字典，整合成一个字典
#可以直接当作DataLoader中的collate_fn函数直接传入
#生成给模型使用的一个字典，要保证键值正确
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels
    return batch

from transformers import DefaultDataCollator
#就是把数据捏合成张量，这个是需要的
data_collator = DefaultDataCollator()

# 评估指标

In [ ]:
#!pip install pycocotools
#!pip install evaluate
import evaluate

metric = evaluate.load("mean_iou")
EvalPrediction(predictions=preds, label_ids=label_ids, inputs=inputs_ids)

In [1]:
评估数据集为 {"pixel_values": pixel_values, "labels": {"image_id": image_id, "annotations": target}}

In [ ]:
import numpy as np
import torch
from torch import nn

def compute_metrics(eval_pred):
    #eval_pred 是一个结果batch，eval模式中，把每个batch结果添加到metric中
    #这是一个中介函数，把真正的计算交给metric
    #这就是这个框架的缺陷，输入就是模型输出，要评估需要在这里做后处理才行
    with torch.no_grad():
        logits_tensor,labels=eval_pred
        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        return metrics

# 模型

In [ ]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer
#model的定义要求  ： forward输出一个大类，必须同时包含loss和prediction
model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint,
                                                         id2label=id2label, 
                                                         label2id=label2id,
                                                        ignore_mismatched_sizes=True).to('cuda')

In [ ]:
#优化器选择
vars(transformers.training_args.OptimizerNames)

# 训练相关的所有参数，没有模型参数

In [4]:
args= TrainingArguments('')

In [1]:
{'output_dir': '',
  'learning_rate': 5e-05,#初始化学习率
  'per_device_train_batch_size': 8,#batch大小,8或64的倍数
  'num_train_epochs': 3.0,#训练次数
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,#学习率变化 
  'optim': <OptimizerNames.ADAMW_HF: 'adamw_hf'>,#优化器 adamw_hf, adamw_torch, adamw_torch_fused, 
                                              # adamw_apex_fused, adamw_anyprecision oradafactor.
                                             #如果不指定，要在trainer中进行实例化
 'optim_args': None,#优化器参数
 'adafactor': False, #是否把adamw替换为adafacotor
 #这些是adamW训练器的参数
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0, 
 'warmup_ratio': 0.0,#参数
 'warmup_steps': 0, 
 
 'bf16': False, #是否用bf16训练模型
 'fp16': False,#是否混合精度训练模型 
  'fp16_opt_level': 'O1',#混合精度apex中的设置参数
 'half_precision_backend': 'auto',# "auto", "cuda_amp", "apex", "cpu_amp" 后端选择，一般用auto就性了
 'bf16_full_eval': False,#bf16评估
 'fp16_full_eval': False,#fp16评估
 'gradient_accumulation_steps': 1,#累加多少步梯度才更新参数
   'gradient_checkpointing': False,#保存梯度用于更新参数
 
  'dataloader_drop_last': False,#最后一个不足batch是否丢掉
 'dataloader_num_workers': 0,#需要多少进程
 'label_smoothing_factor': 0.0,#标签平滑的系数，0就是不用 【0，1】
 
 'evaluation_strategy': <IntervalStrategy.NO: 'no'>,#评估是按epoch还是iter,或者不评估[no,steps,epoch]
 'per_device_eval_batch_size': 8,

 'eval_accumulation_steps': None,#预测生成多少步才把结果搬运到CPU上
 'eval_delay': 0,  #第一步评估开始前，执行训练多少次，推迟评估的作用
 
  'logging_steps': 500,#记录
  'logging_dir': 'runs\\Dec02_22-12-45_DESKTOP-RJGFSP1',#日志保存路径
 
 'save_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'save_steps': 500,#保存步数
 'save_total_limit': None,
 'save_safetensors': False,
 'save_on_each_node': False,

 'seed': 42,
 'data_seed': None,#数据采样的种子
  'dataloader_pin_memory': True,#pin_memory设置
 'resume_from_checkpoint': None,#继续训练，传入文件
 
 
 'jit_mode_eval': False,#用jit格式的模型进行推断
 'use_ipex': False,#Intel extension 优化pytorch的方法，是否使用
 
 'tf32': None,#tf32格式
 'local_rank': 0,#分布式训练参数
 'ddp_backend': None,#"nccl"`, `"mpi"`, `"ccl"`, `"gloo"后端选择
 'sharded_ddp': [],#用Sharded DDP training 参数为simple"，zero_dp_2，zero_dp_3，offload
 'fsdp': [],#pytorch分布式训练  ，full_shard，shard_grad_op，offload，auto_wrap
  'ddp_find_unused_parameters': None,#分布式训练中find_unused_parameters参数
 'ddp_bucket_cap_mb': None,#分布式训练中find_unused_parameters参数
  'ddp_timeout': 1800,
 'fsdp_config': {'fsdp_min_num_params': 0,
  'xla': False,
  'xla_fsdp_grad_ckpt': False},#这里也有许多参数可以使用
 
 'deepspeed': None,#是否使用deepspeed
 'report_to': ['tensorboard', 'wandb'],#后端的支持有很多，'all'
 
 'include_inputs_for_metrics': False,#是否把输入传递给merics


 'torch_compile': True,#编译模型,如果版本不对,用torch._dynamo.config.suppress_errors = True
 'overwrite_output_dir': False,#是否修改保存路径
'max_steps': -1,#会覆盖num_train_epochs 的训练次数，一样的
 
 #日志相关参数
  'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': False,
  'logging_nan_inf_filter': True,
  'tpu_num_cores': None,#如果在TPU上训练，需要设置核心个数
 'tpu_metrics_debug': False,
 'debug': [],
  'eval_steps': None,#
  'past_index': -1,#一些模型用于输出的层
  'run_name': '',#wandb要用的
  'disable_tqdm': False,#显示训练过程
  'remove_unused_columns': True,#是否去掉?
  'label_names': None,
 
  'load_best_model_at_end': False,
  'metric_for_best_model': None,
  'greater_is_better': None,
 
 'ignore_data_skip': False,#继续训练会快点
  'group_by_length': False,#
  'length_column_name': 'length',
  'skip_memory_metrics': True,#一般为Ture，不然会减慢训练和评估速度
  'use_legacy_prediction_loop': False,
  'push_to_hub': False, # 恶心的设置
  'hub_model_id': None,
 'hub_strategy': <HubStrategy.EVERY_SAVE: 'every_save'>,
 'hub_token': None,
 'hub_private_repo': False,
 'prediction_loss_only': False,
 'push_to_hub_model_id': None,
 'push_to_hub_organization': None,
 'push_to_hub_token': None,
  'mp_parameters': '',#sagmaker
  'full_determinism': False,#debug才会用这个
  'ray_scope': 'last',#搜索超参数
  'do_train': False,  #  是否训练
 'do_eval': False,  #    是否评估   这个三个参数会自动设置
 'do_predict': False,#是否预测
  'auto_find_batch_size': False,#当CUDA内存不够，把batch大小减少一半
}

SyntaxError: expression expected after dictionary key and ':' (4032607840.py, line 3)

In [ ]:
#优化器选择
"adamw_hf"，"adamw_torch"，"adamw_torch_fused"，
"adamw_torch_xla"，"adamw_apex_fused"，
"adafactor"，"adamw_anyprecision"， "sgd"，"adagrad"，
"adamw_bnb_8bit"，"adamw_8bit"  ，"lion_8bit"，
"lion_32bit"，"paged_adamw_32bit"， "paged_adamw_8bit"，
"paged_lion_32bit"， "paged_lion_8bit"，

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    
    per_device_train_batch_size=64,
    
    warmup_ratio=0.1,
    optimizers= (optimizer,None),  # optimizer=torch.optim.SGD(model.parameters(),lr=1e-5)
    
    gradient_accumulation_steps=1,
    
    #evaluate，可以直接去掉
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    eval_steps=5,
    
    fp16=True,
    optim='adafactor',
    dataloader_pin_memory =True,
    dataloader_num_workers=2,
    
    logging_steps=10,
    
    save_steps=5,
    report_to="tensorboard",
    metric_for_best_model="accuracy",
    remove_unused_columns=False,#这个是先删除没用的再后处理，以一定要注意，只能去删除哪些一定不会用的
    
    label_names=["labels"],#这个必须给，不然容易报错label,label_id都被占用了，labels特指分类标签，会用label_smooth来处理

    auto_find_batch_size=False,#当CUDA内存不够，把batch大小减少一半
)


# deepspeed
    git clone https://github.com/microsoft/DeepSpeedExamples
    cd DeepSpeedExamples
    find . -name '*json'
    # find examples with the Lamb optimizer
    grep -i Lamb $(find . -name '*json')

In [ ]:
TrainingArguments(..., deepspeed="path/to/deepspeed_config.json")

# 训练器

In [1]:
#可以使用apex
trainer = Trainer(
    model=model,#直接是模型就可以
    model_init= None,#和model两个参数必须给一个
    args=training_args,# 参数，可以是空，默认有
    data_collator=data_collator,#可以设为空，这是有默认default_data_collator处理字典数据的
    train_dataset=mnist["train"],
    tokenizer=image_processor,#用于保存的，
    
    #eval_dataset=mnist["test"],
    #compute_metrics=compute_metrics,#评估函数
    tokenizer= None,#文本可能会用到
    callbacks= None,#钩子，有默认的，如果给了就是和默认的组合在一起所有的
    optimizers= (None, None),#优化器和学习率
    preprocess_logits_for_metrics= None,
)

trainer.train(
    #resume_from_checkpoint='/kaggle/working/detr-resnet-50_finetuned_cppe5/checkpoint-200/'
)#训练是训练，评估是评估

SyntaxError: keyword argument repeated: per_device_train_batch_size (3791893107.py, line 8)

In [ ]:
trainer中的compute_loss(self, model, inputs, return_outputs=False)就是用

    model(**inputs)的方法计算损失{'loss':value}的，所以模型执行必须是能计算损失的
    在这调用loss函数

In [ ]:
trainer.evaluate(resume_from_checkpoint='/kaggle/working/detr-resnet-50_finetuned_cppe5/checkpoint-200/')

# LORA训练

In [1]:
from peft import LoraConfig,get_peft_model

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
#用法,所有重要参数保存，LoRA
from peft import LoraConfig, TaskType,get_peft_model

#################自定义模型中使用peft######################
#1定义模型，是一个nn.module就行了
class Mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,inputs):
        return 
#根据每一层的名字，来确定具体对
print([(n, type(m)) for n, m in Mymodel().named_modules()])
peft_config = LoraConfig(target_modules=["layer1", "layer2"],#想分解替换的，不能是最后一层，不然输出变了
                         modules_to_save=["layer3"],#想依然训练的,保留最后一层
                         r=8, 
                         lora_alpha=32,
                         lora_dropout=0.1)

#only `torch.nn.Linear` and `Conv1D` are supported.
model = MLP()
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()
# prints trainable params: 56,164 || all params: 4,100,164 || trainable%: 1.369798866581922

#训练参数中，可以使用更大的Batch，和更大的学习率

In [ ]:
#新的方式是这样用的
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    modules_to_save=["lm_head"],
)

model.add_adapter(lora_config)

In [ ]:
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    '''
    继承trainer，需要修改里面的 compute_loss(self, model, inputs, return_outputs=False)
    返回一个loss标量
    '''
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# 优化训练

In [ ]:
#Using 🤗 PEFT

#使用acclerator
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

dataloader = DataLoader(ds, batch_size=training_args.per_device_train_batch_size)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(fp16=training_args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
#用多专家模型

#torch.nn.functional.scaled_dot_product_attention (SDPA)
#FLashattention

# 多GPU 训练优化
    
    
    1模型可以放入单个GPU时
        可以使用DDP或者ZERO
    2当模型不能放入单个GPU时，太大了
        用PP,ZERO,TP
    3模型最大的层放不进一个GPU
        TP
        
     DP和DPP的区别，DP是用GPU0来执行归一化和任务分发 
     1DPP每个batch只通信一次，DDP有5个通信，而且DP用GIL
     2DP中GPU0有大量工作要做
     3DDP支持跨机器的分布式训练，DP不行
    